# Ensemble Techniques

## Lab 12
## 2012006

Importing libraries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

Import data, df is for operations and data is for final result appends

In [2]:
df = pd.read_csv("/content/ensembe_train.csv") #operating
data = pd.read_csv("/content/ensembe_train.csv") #appending

Looking at the data

In [3]:
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


We can see that url_legal and license have NaN, let us see the amount of null

In [4]:
df.isna().sum() #null count

id                   0
url_legal         2004
license           2004
excerpt              0
target               0
standard_error       0
dtype: int64

Looking at the description of all columns, we can see that the two columns have heavy amount of nulls, hence we can drop it.

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
target,2834.0,-0.959319,1.033579,-3.676268,-1.690320,-0.912190,-0.202540,1.711390
standard_error,2834.0,0.491435,0.034818,0.000000,0.468543,0.484721,0.506268,0.649671


Taking the word count of the column ```excerpt```

In [20]:
s = data['excerpt'].str.split(expand=True).stack().value_counts()

Looking at the shape

In [6]:
df.shape

(2834, 6)

Dropping the columns

In [8]:
df = df.drop(['url_legal','license'], axis = 1)

Putting the data into two separate frames, for model running

In [9]:
target = df['target']
train = df['excerpt']

df = df.drop(['target','excerpt'],axis=1)

Looking at the head

In [10]:
df.head()

,id,standard_error
0,c12129c31,0.464009
1,85aa80a4c,0.480805
2,b69ac6792,0.476676
3,dd1000b26,0.450007
4,37c1b32fb,0.510845


Since, Random forest cannot run on text data, we will encode the text data so that the regressor runs.

In [11]:
le = preprocessing.LabelEncoder()
le.fit(train)
train = le.transform(train)
train= le.fit_transform(train)

Looking at the encoded label

In [12]:
train

array([2743,  291,  390, ..., 2295, 1839,  369])

Since, the regressor takes 2D values and the train is 1D, that is why we need to reshape the column as 2D and then feed into regressor

In [13]:
train = train.reshape(-1,1) #reshape

The model:

In [14]:
#model with 500 estimators
clf = RandomForestRegressor(n_estimators = 500)

# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(train, target)

# performing predictions on the test dataset
pred = clf.predict(train)

In [15]:
#changing to datafram
pred = pd.DataFrame(pred) 

In [16]:
#appending the column
data['RF-LabelEncoding'] = pred

# Adaboost

In [17]:
#making random forest regressor
RFR=RandomForestRegressor(n_estimators = 500)

#applying adaboost
RegModel = AdaBoostRegressor(n_estimators=50, base_estimator=RFR ,learning_rate=1)

#training on the data
AB=RegModel.fit(train, target)

#taking results
pred=AB.predict(train)

In [18]:
#converting to dataframe
pred = pd.DataFrame(pred)

#appending
data['AB-LabelEncoding'] = pred

# Testing

In [30]:
test = pd.read_csv('/content/ensemble_test.csv')

In [33]:
test_train = test['excerpt']
le = preprocessing.LabelEncoder()
le.fit(test_train)
test_train = le.transform(test_train)
test_train= le.fit_transform(test_train)

In [36]:
test_train = test_train.reshape(-1,1)

In [37]:
#making prediction from Bagging
predRF = clf.predict(test_train)

#making prediction from Boosting
predAB = AB.predict(test_train)

In [38]:
predRF = pd.DataFrame(predRF)
predAB = pd.DataFrame(predAB)

In [39]:
test['target-RF'] = predRF
test['target-AB'] = predAB

# Dataframe

In [54]:
#making dataframe
output = test.to_csv('output.csv',index=False)